In [ ]:
import rdflib
import html
g = rdflib.Graph().parse('./fb15k-wikitop2021-yago-facts.nt', format='ntriples')
g += rdflib.Graph().parse('./fb15k-wikitop2021-yago-full-types.nt', format='ntriples')

from rdflib.namespace import RDF
for line in open('fb15k-wikitop2021.tsv').readlines()[1:]:
    subject = rdflib.URIRef( line.split('\t')[6][1:-1] ) # take the URI in column 6
    g.add( (subject, RDF.type, rdflib.URIRef('http://example.com/popularEntity')) )

In [ ]:
r = g.query("""
select (count(*) as ?count) ?p ?o  where 
{
    ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    ?s ?p ?o .
} 
group by ?p ?o 
order by desc(?count)
limit 10
""")
for b in r.bindings:
    print(*[f'{x.split("/")[-1][:40]:40s}' for x in b.values()])

## Code

In [ ]:
%load_ext ipython_sparql_pandas

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper('http://localhost:7200/repositories/20qs')
sparql.setReturnFormat(JSON)

In [ ]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
def extractCountAndAttr(result):
    listOfSplits = []
    listOfObj = []
    listOfPred= []
    #ret = result.queryAndConvert()
    for attr in result["results"]["bindings"]:
        listOfSplits.append(int(attr['count']["value"]))
        listOfObj.append(attr['o']["value"])
        listOfPred.append(attr['p']["value"])
    return(listOfSplits,listOfObj,listOfPred)

#function to approximate index element of list that provides best split
def bestSplit(listOfNumbers, ElementsLeft):
    currentBest=1
    indexBest=-1
    for i in range(len(listOfNumbers)):
        splitScore = abs((listOfNumbers[i]/ElementsLeft)-0.5)
        if splitScore<currentBest:
            currentBest=splitScore
            indexBest=i
    return indexBest
"""
def generateQuestion(listOfPred, listOfObj, index):
    userAnswer = input(f'Does the thing you are looking for have the attribute: {listOfPred[index].split("/")[-1]} {listOfObj[index].split("/")[-1]} ?' ).startswith('y')
    if userAnswer:
        PosAttr.append('?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>.' )   
    else:
        NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>. }}' )   
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)"""


def generateQuestion(listOfPred, listOfObj, index):
    userAnswer = input(f'Does the thing you are looking for have the attribute: {listOfPred[index].split("/")[-1]} {listOfObj[index].split("/")[-1]} ?' )
    if userAnswer.startswith('y'):
        PosAttr.append('?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>.' )   
    elif userAnswer.startswith('n'):
        NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>. }}' )   
    else:
        generateQuestion(listOfPred, listOfObj, index+1)
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)


  
def updateQuery( PosFilters, NegFilters):

    FilterQuestion =  f"""
            select (count(*) as ?count) ?p ?o  where 
            {{
            
            {PosFilters}
             ?s ?p ?o .
            {NegFilters}
            }} 
            group by ?p ?o 
            order by desc(?count)
            limit 25
            """
        
    return FilterQuestion

In [ ]:
# resets are variables
def game_reset():
    FilterQuestion =  f"""
        select (count(*) as ?count) ?p ?o  where 
        {{

        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
         ?s ?p ?o .
        }} 
        group by ?p ?o 
        order by desc(?count)
        limit 25
        """
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()


    PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
    NegAttr=[]
    availableSplits, listOfObj ,listOfPred = extractCountAndAttr(qres)
    index= bestSplit(availableSplits, availableSplits[0])
    return (qres, PosAttr, NegAttr)

In [ ]:
#without printing results
qres,PosAttr,NegAttr = game_reset()
i=0
while i< 20:
   
    availableSplits, listOfObj ,listOfPred = extractCountAndAttr(qres)
    if availableSplits[0]<=1:
        break
    index= bestSplit(availableSplits, availableSplits[0])
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, index)
    query = updateQuery(PosFilters, NegFilters)
    sparql.setQuery(query)
    qres = sparql.queryAndConvert()
    

finalResult= f"""
            select distinct ?s
            where 
            {{
            
            {PosFilters}
            ?s ?p ?o
            {NegFilters}
            }} """
sparql.setQuery(finalResult)
finalRes = sparql.queryAndConvert()
for attr in finalRes["results"]["bindings"]:
    print(*[f'Your Answer is: {x.split("/")[-1]}' for x in [attr['s']['value']]][0:10]) 

In [ ]:
# with count print
qres,PosAttr,NegAttr = game_reset()
i=0
while i< 20:
    availableSplits, listOfObj ,listOfPred = extractCountAndAttr(qres)
    if availableSplits[0]<=1:
        break
    index= bestSplit(availableSplits, availableSplits[0])
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, index)
    query = updateQuery(PosFilters, NegFilters)
    sparql.setQuery(query)
    qres = sparql.queryAndConvert()
    for r in qres["results"]["bindings"]:
        print(*[f'{x.split("/")[-1][:40]:40s}' for x in [r['count']["value"],r['p']["value"],r['o']["value"]]])

finalResult= f"""
            select distinct ?s
            where 
            {{
            
            {PosFilters}
            ?s ?p ?o
            {NegFilters}
            }} """
sparql.setQuery(finalResult)
finalRes = sparql.queryAndConvert()
for attr in finalRes["results"]["bindings"]:
    print(*[f'Your Answer is: {x.split("/")[-1]}' for x in [attr['s']['value']]][0:10]) 

In [ ]:
# with guess print
def guess(PosFilters, NegFilters):
    FilterQuestion =  f"""
            select ?s  where 
            {{
            {PosFilters}
             ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters}
            }}  
            order by desc(?count)
            limit 25
            """
        
    return FilterQuestion

i=0
qres,PosAttr,NegAttr = game_reset()
while i< 20:
   
    availableSplits, listOfObj ,listOfPred = extractCountAndAttr(qres)
    if availableSplits[0]<=1:
        break
    index= bestSplit(availableSplits, availableSplits[0])
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, index)
    query = updateQuery(PosFilters, NegFilters)
    sparql.setQuery(query)
    qres = sparql.queryAndConvert()
    guesses = guess(PosFilters, NegFilters)
    sparql.setQuery(guesses)
    left = sparql.queryAndConvert()
    print('Possible Guesses:')
    for subject in left['results']['bindings']:
        print(*[f'{x.split("/")[-1]}' for x in [subject['s']['value']]][0:10]) 
    print()
    

finalResult= f"""
            select distinct ?s
            where 
            {{
            
            {PosFilters}
            ?s ?p ?o
            {NegFilters}
            }} """
sparql.setQuery(finalResult)
finalRes = sparql.queryAndConvert()
for attr in finalRes["results"]["bindings"]:
    print(*[f'Your Answer is: {x.split("/")[-1]}' for x in [attr['s']['value']]][0:10]) 